In elixir, **all code runs inside processes**. Processes are isolated from each other, run concurrent to one another and communicate via message passing. Processes are not only the basis for concurrency in Elixir, but they aslo provide the means for building distributed and fault-tolerant programs.

Elixir's processes should not be confused with operating system processes. Processes in elixir are extrememly lightweight in terms of memory and CPU. Because of this, it is not uncommon to have tens or even hundreds of thousands of processes running simultaneously.

```elixir
spawn fn -> 1 + 2 end
```

```elixir
iex|2 ▶ pid = spawn fn -> 1 + 2 end
#PID<0.137.0>
iex|3 ▶ Process.alive?(pid)
false
```

1. Elixir is an immutable language where nothing is shared by default. If we want to provide buckets, which can be read and modified from multiple places, we have two main options in Elixir:

- Processes
- ETS (Erlang Term Storage)

- **Agent**: Simple wrappers around state
- **GenServer**: "Generic servers"(processes) that encapsulate state, provide sync and async calls, support code reloading, and more. 
- 


In [1]:
self()

#PID<0.209.0>

In [2]:
Process.alive?(self())

true

We can send messages to a process with `send/2` and receive them with `receive/1`

In [3]:
send self(), {:hello, "world"}

{:hello, "world"}

In [ ]:
receive do
 {:hello, msg} -> msg
 {:world, msg} -> "won't match"
end

When a message is sent to a process, the message is stored in the process mailbox. The `receive/1` block goes through the current process mailbox searching for a message that matches any of the given patterns.`receive/1` supports guards and many clauses, such as `case/2`

The process that sends the message does not block on `send/2`, it puts the message in the recipient's mailbox and continues. In particular, a process can send message to itself.

If there is no message in the mailbox matching any of the patterns, the current process will wait until a matching message arrives. A timeout can also be specified:

In [1]:
receive do
  {:hello, msg} -> msg
after
  1_000 -> "nothing after 1s"
  end

"nothing after 1s"

In [2]:
parent = self()
spawn fn -> send(parent, {:hello, self()}) end
receive do
  {:hello, pid} -> "Got hello from #{inspect pid}"
end


"Got hello from #PID<0.224.0>"

`flush/0` flushes and prints all the messages in the mailbox

In [3]:
send self(), :hello

:hello

In [4]:
flush()

:ok

In [5]:
spawn fn -> raise "oops" end

#PID<0.229.0>

[error] Process #PID<0.58.00> raised an exception
** (RuntimeError) oops
    (stdlib) erl_eval.erl:668: :erl_eval.do_apply/6

It merely logged an error but the parent process is still running. That's because processes are isolated. If we want the failure in one process to propagate to another one, we should link them. This can be done with `spawn_link/1`

In [6]:
self()

#PID<0.209.0>

In [ ]:
spawn_link fn -> raise "oops" end

Because processes are linked, we now see a message saying the parent process, which is the shell process, has received an EXIT signal from another process causing the shell to terminate. IEx detects this situation and starts a new shell session

Processes and links play an important role when building fault-tolerant systems. **Elixir processes are isolated and don't share anything by defaut**. Therefore, a failure in a process will never crash or corrupt the state of another process. Links, however, allow proceses to establish a relationship in a case of failures. We often link our processes to supervisors which will detect when a process dies and start a new process in its place.

While other languages would require us to catch/handle exceptions, in Elixir we are actually fine with letting processes fail because we expect supervisors to properly restart our systems. "Failing fast" is a common philosophy when writing Elixir software.